In [ ]:

!pip install torch torchvision scikit-learn pillow

In [ ]:
import os
import random
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
# Dataset paths
EYE_TEST_DIR = "dataset_split/eyes/test"
MOUTH_TEST_DIR = "dataset_split/mouth/test"

# Model paths
EYE_MODEL_PATH = "eye_state_efficientnet_b0.pth"
MOUTH_MODEL_PATH = "mouth_state_efficientnet_b0.pth"


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
eye_model = models.efficientnet_b0(weights=None)
eye_model.classifier[1] = nn.Linear(
    eye_model.classifier[1].in_features, 1
)

eye_model.load_state_dict(
    torch.load(EYE_MODEL_PATH, map_location=device)
)

eye_model.eval().to(device)
print("Eye-state model loaded")

Eye-state model loaded


In [ ]:
mouth_model = models.efficientnet_b0(weights=None)
mouth_model.classifier[1] = nn.Linear(
    mouth_model.classifier[1].in_features, 1
)

mouth_model.load_state_dict(
    torch.load(MOUTH_MODEL_PATH, map_location=device)
)

mouth_model.eval().to(device)
print("Mouth-state model loaded")

Mouth-state model loaded


In [ ]:
def predict_prob(model, image_path):
    img = Image.open(image_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        prob = torch.sigmoid(model(img)).item()

    return prob

In [ ]:
def fuse_alertness(eye_prob, mouth_prob):
    """
    eye_prob   > 0.5 → open
    mouth_prob > 0.5 → yawn

    Returns:
    0 → ALERT
    1 → NOT ALERT
    """
    eye_open = eye_prob > 0.5
    mouth_yawn = mouth_prob > 0.5

    if eye_open and not mouth_yawn:
        return 0  # ALERT
    else:
        return 1  # NOT ALERT

In [ ]:
random.seed(42)

eye_open = os.listdir(f"{EYE_TEST_DIR}/open")
eye_closed = os.listdir(f"{EYE_TEST_DIR}/closed")

mouth_no_yawn = os.listdir(f"{MOUTH_TEST_DIR}/no_yawn")
mouth_yawn = os.listdir(f"{MOUTH_TEST_DIR}/yawn")

pairs = []

# ALERT pairs (open + no_yawn)
for eye in random.sample(eye_open, min(len(eye_open), len(mouth_no_yawn))):
    mouth = random.choice(mouth_no_yawn)
    pairs.append(("open", eye, "no_yawn", mouth, 0))

# NOT ALERT pairs (closed + yawn)
for eye in random.sample(eye_closed, min(len(eye_closed), len(mouth_yawn))):
    mouth = random.choice(mouth_yawn)
    pairs.append(("closed", eye, "yawn", mouth, 1))

random.shuffle(pairs)

print("Total synthetic test pairs:", len(pairs))


Total synthetic test pairs: 219


In [ ]:
y_true = []
y_pred = []

for eye_cls, eye_img, mouth_cls, mouth_img, label in pairs:

    eye_path = f"{EYE_TEST_DIR}/{eye_cls}/{eye_img}"
    mouth_path = f"{MOUTH_TEST_DIR}/{mouth_cls}/{mouth_img}"

    eye_prob = predict_prob(eye_model, eye_path)
    mouth_prob = predict_prob(mouth_model, mouth_path)

    pred = fuse_alertness(eye_prob, mouth_prob)

    y_true.append(label)
    y_pred.append(pred)

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=["ALERT", "NOT ALERT"]
))

Confusion Matrix:
[[ 85  25]
 [  0 109]]

Classification Report:
              precision    recall  f1-score   support

       ALERT       1.00      0.77      0.87       110
   NOT ALERT       0.81      1.00      0.90       109

    accuracy                           0.89       219
   macro avg       0.91      0.89      0.88       219
weighted avg       0.91      0.89      0.88       219

